In [ ]:
#일단 PDF에 나와있는대로 코드 적은 결과(대부분의 코드가 PDF 강의 자료 기준으로 
#그대로 쓴 것이 많음)

In [ ]:
#csv파일로 저장까지

In [ ]:
#근데 예측확률값(pred_prob)가 정확히 뭔지는 모르겠습니다.

In [ ]:
#그래서 투표하는 확률을 넣었습니다.
#제가 쓴 기준으로는 LGBM이 가장 F1이 높게 나오네요.

In [15]:
#Decision Tree 결과
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import warnings
import xgboost as xgb
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from sklearn.datasets import load_breast_cancer
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

train2=pd.read_csv("train2.csv")
test=pd.read_csv("test2.csv")

zero_features = ['education', 'engnat', 'hand', 'married']

index = test['index']

pd.set_option('display.max_columns', 100)

Unneed_list = ['QaE', 'QbE', 'QcE', 'QdE', 'QeE','QfE', 'QgE', 'QhE', 'QiE', 'QjE',
               'QkE', 'QlE', 'QmE', 'QnE', 'QoE','QpE', 'QqE', 'QrE', 'QsE', 'QtE', 'index']
train2.drop(Unneed_list, axis=1, inplace=True)
test.drop(Unneed_list, axis=1, inplace=True)


train3=pd.get_dummies(train2)
test=pd.get_dummies(test)


zero_features = ['education', 'engnat', 'hand', 'married']

#결손값(무응답 = 0)제거 함수
def drop_NoResponse(df):
    for col in zero_features:
        df.drop(df[df[col]==0].index, axis=0, inplace=True)
        
    return df
train = drop_NoResponse(train3)



max_index = train['familysize'].idxmax()
train.drop(max_index, axis=0, inplace=True)
#trainX에 투표값 제거
train_X = train.drop('voted', axis = 1)

#trainy에 투표값만 넣음
train_y = train['voted']


X_train, X_test, y_train, y_test=train_test_split(train_X, train_y, test_size=0.2, random_state=0)
dt_clf = DecisionTreeClassifier(random_state=0)
parameters = {'max_depth':[2, 3, 4, 5, 6, 7, 8, 9, 10], 'min_samples_split':[2, 4, 6, 7, 8, 10, 12, 14]}
grid_dtree = GridSearchCV(dt_clf, param_grid=parameters, cv=5, refit=True)
grid_dtree.fit(X_train, y_train)

estimator = grid_dtree.best_estimator_
pred = estimator.predict(X_test)

dt_pred = grid_dtree.predict(X_test)

#하이퍼 파라미터 확인
scores_df=pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score','split0_test_score','split1_test_score','split2_test_score']]

print('GridSearchCV 최적 하이퍼 파라미터 : ', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dtree.best_score_))
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))
get_clf_eval(y_test, dt_pred)

X_new = test
X_new.shape
new_pred_class = grid_dtree.predict(X_new)
pred_proba = grid_dtree.predict_proba(X_new)[:,1]
new_pred_class.shape
print(pred_proba)

sub = pd.DataFrame({'index':index, 'pred_proba':pred_proba, 'pred': new_pred_class})
sub.set_index('index', inplace=True)
sub.to_csv('decision_sum.csv')

GridSearchCV 최적 하이퍼 파라미터 :  {'max_depth': 5, 'min_samples_split': 7}
GridSearchCV 최고 정확도: 0.6872
테스트 데이터 세트 정확도: 0.6943
오차행렬
[[2349 1541]
 [ 644 2613]]
정확도: 0.6943, 정밀도: 0.6290, 재현율: 0.8023, F1: 0.7052
[0.51860078 0.72411863 0.22149208 ... 0.4774359  0.09394314 0.69732938]


In [3]:
#voting  결과
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import warnings
import xgboost as xgb
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from sklearn.datasets import load_breast_cancer
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

train2=pd.read_csv("train2.csv")
test=pd.read_csv("test2.csv")

zero_features = ['education', 'engnat', 'hand', 'married']
index = test['index']

pd.set_option('display.max_columns', 100)

Unneed_list = ['QaE', 'QbE', 'QcE', 'QdE', 'QeE','QfE', 'QgE', 'QhE', 'QiE', 'QjE',
               'QkE', 'QlE', 'QmE', 'QnE', 'QoE','QpE', 'QqE', 'QrE', 'QsE', 'QtE', 'index']
train2.drop(Unneed_list, axis=1, inplace=True)
test.drop(Unneed_list, axis=1, inplace=True)


train3=pd.get_dummies(train2)
test=pd.get_dummies(test)


zero_features = ['education', 'engnat', 'hand', 'married']

#결손값(무응답 = 0)제거 함수
def drop_NoResponse(df):
    for col in zero_features:
        df.drop(df[df[col]==0].index, axis=0, inplace=True)
        
    return df
train = drop_NoResponse(train3)



max_index = train['familysize'].idxmax()
train.drop(max_index, axis=0, inplace=True)
#trainX에 투표값 제거
train_X = train.drop('voted', axis = 1)

#trainy에 투표값만 넣음
train_y = train['voted']




lr_clf = LogisticRegression()
knn_clf =KNeighborsClassifier(n_neighbors=8)

vo_clf = VotingClassifier(estimators=[('LR',lr_clf),('KNN', knn_clf)], voting='soft')

X_train, X_test, y_train, y_test=train_test_split(train_X, train_y, test_size=0.2, random_state=0)


vo_clf.fit(X_train, y_train)
pred = vo_clf.predict(X_test)
print('보팅 분류기 정확도:{0:.4f}'.format(accuracy_score(y_test, pred)))
classifiers = [lr_clf, knn_clf]
for classifier in classifiers:
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    class_name=classifier.__class__.__name__
    print('{0} 정확도:{1:.4}'.format(class_name, accuracy_score(y_test, pred)))



X_new = test
X_new.shape
new_pred_class = vo_clf.predict(X_new)
pred_proba = vo_clf.predict_proba(X_new)[:,1]
new_pred_class.shape
print(pred_proba)

sub = pd.DataFrame({'index':index, 'pred_proba':pred_proba, 'pred': new_pred_class})
sub.set_index('index', inplace=True)
sub.to_csv('voting_sum.csv')





보팅 분류기 정확도:0.6758
LogisticRegression 정확도:0.6929
KNeighborsClassifier 정확도:0.6079
[0.33341376 0.67345905 0.34528383 ... 0.46764549 0.24294696 0.48384517]


In [10]:
#XGBOOST 결과
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import warnings
import xgboost as xgb
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from sklearn.datasets import load_breast_cancer
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

train2=pd.read_csv("train2.csv")
test=pd.read_csv("test2.csv")

zero_features = ['education', 'engnat', 'hand', 'married']

index = test['index']

pd.set_option('display.max_columns', 100)

Unneed_list = ['QaE', 'QbE', 'QcE', 'QdE', 'QeE','QfE', 'QgE', 'QhE', 'QiE', 'QjE',
               'QkE', 'QlE', 'QmE', 'QnE', 'QoE','QpE', 'QqE', 'QrE', 'QsE', 'QtE', 'index']
train2.drop(Unneed_list, axis=1, inplace=True)
test.drop(Unneed_list, axis=1, inplace=True)


train3=pd.get_dummies(train2)
test=pd.get_dummies(test)


zero_features = ['education', 'engnat', 'hand', 'married']

#결손값(무응답 = 0)제거 함수
def drop_NoResponse(df):
    for col in zero_features:
        df.drop(df[df[col]==0].index, axis=0, inplace=True)
        
    return df
train = drop_NoResponse(train3)



max_index = train['familysize'].idxmax()
train.drop(max_index, axis=0, inplace=True)
#trainX에 투표값 제거
train_X = train.drop('voted', axis = 1)

#trainy에 투표값만 넣음
train_y = train['voted']


X_train, X_test, y_train, y_test=train_test_split(train_X, train_y, test_size=0.2, random_state=0)


xgb_wrapper = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
evals=[(X_test, y_test)]
xgb_wrapper.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='logloss',
               eval_set=evals, verbose=True)
w_preds = xgb_wrapper.predict(X_test)
w_preds_proba = xgb_wrapper.predict_proba(X_test)[:, 1]

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))
    
get_clf_eval(y_test, w_preds)

X_new = test
X_new.shape
new_pred_class = xgb_wrapper.predict(X_new)
new_pred_class.shape
pred_proba = xgb_wrapper.predict_proba(X_new)[:,1]
sub = pd.DataFrame({'index':index, 'pred_prob':pred_proba, 'pred': new_pred_class})
sub.set_index('index', inplace=True)
sub.to_csv('xgb2_sum.csv')

[0]	validation_0-logloss:0.673674
Will train until validation_0-logloss hasn't improved in 100 rounds.
[1]	validation_0-logloss:0.65773
[2]	validation_0-logloss:0.644432
[3]	validation_0-logloss:0.633442
[4]	validation_0-logloss:0.623905
[5]	validation_0-logloss:0.615989
[6]	validation_0-logloss:0.609275
[7]	validation_0-logloss:0.603172
[8]	validation_0-logloss:0.598274
[9]	validation_0-logloss:0.593942
[10]	validation_0-logloss:0.590069
[11]	validation_0-logloss:0.586857
[12]	validation_0-logloss:0.583934
[13]	validation_0-logloss:0.581239
[14]	validation_0-logloss:0.578949
[15]	validation_0-logloss:0.576808
[16]	validation_0-logloss:0.575004
[17]	validation_0-logloss:0.573632
[18]	validation_0-logloss:0.572267
[19]	validation_0-logloss:0.570988
[20]	validation_0-logloss:0.569923
[21]	validation_0-logloss:0.569105
[22]	validation_0-logloss:0.568293
[23]	validation_0-logloss:0.56751
[24]	validation_0-logloss:0.566863
[25]	validation_0-logloss:0.566085
[26]	validation_0-logloss:0.56535

[230]	validation_0-logloss:0.554453
[231]	validation_0-logloss:0.554483
[232]	validation_0-logloss:0.554416
[233]	validation_0-logloss:0.55435
[234]	validation_0-logloss:0.554336
[235]	validation_0-logloss:0.554324
[236]	validation_0-logloss:0.554316
[237]	validation_0-logloss:0.554277
[238]	validation_0-logloss:0.554287
[239]	validation_0-logloss:0.554318
[240]	validation_0-logloss:0.554334
[241]	validation_0-logloss:0.554313
[242]	validation_0-logloss:0.554381
[243]	validation_0-logloss:0.554399
[244]	validation_0-logloss:0.554372
[245]	validation_0-logloss:0.554359
[246]	validation_0-logloss:0.554377
[247]	validation_0-logloss:0.554365
[248]	validation_0-logloss:0.554335
[249]	validation_0-logloss:0.554378
[250]	validation_0-logloss:0.554373
[251]	validation_0-logloss:0.554357
[252]	validation_0-logloss:0.554342
[253]	validation_0-logloss:0.554364
[254]	validation_0-logloss:0.554412
[255]	validation_0-logloss:0.554409
[256]	validation_0-logloss:0.554446
[257]	validation_0-logloss:0.

In [17]:
#LGBM 결과
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import warnings
import xgboost as xgb
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from sklearn.datasets import load_breast_cancer
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

train2=pd.read_csv("train2.csv")
test=pd.read_csv("test2.csv")

zero_features = ['education', 'engnat', 'hand', 'married']

index = test['index']

pd.set_option('display.max_columns', 100)

Unneed_list = ['QaE', 'QbE', 'QcE', 'QdE', 'QeE','QfE', 'QgE', 'QhE', 'QiE', 'QjE',
               'QkE', 'QlE', 'QmE', 'QnE', 'QoE','QpE', 'QqE', 'QrE', 'QsE', 'QtE', 'index']
train2.drop(Unneed_list, axis=1, inplace=True)
test.drop(Unneed_list, axis=1, inplace=True)


train3=pd.get_dummies(train2)
test=pd.get_dummies(test)


zero_features = ['education', 'engnat', 'hand', 'married']

#결손값(무응답 = 0)제거 함수
def drop_NoResponse(df):
    for col in zero_features:
        df.drop(df[df[col]==0].index, axis=0, inplace=True)
        
    return df
train = drop_NoResponse(train3)



max_index = train['familysize'].idxmax()
train.drop(max_index, axis=0, inplace=True)
#trainX에 투표값 제거
train_X = train.drop('voted', axis = 1)

#trainy에 투표값만 넣음
train_y = train['voted']


X_train, X_test, y_train, y_test=train_test_split(train_X, train_y, test_size=0.2, random_state=0)

lgbm_wrapper = LGBMClassifier(n_estimators=400)

evals = [(X_test, y_test)]
lgbm_wrapper.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='logloss',
                eval_set=evals, verbose=True)
preds = lgbm_wrapper.predict(X_test)


def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))
    
get_clf_eval(y_test, preds)
X_new = test
X_new.shape
new_pred_class = lgbm_wrapper.predict(X_new)
new_pred_class.shape
pred_proba = lgbm_wrapper.predict_proba(X_new)[:,1]
sub = pd.DataFrame({'index':index, 'pred_prob':pred_proba, 'pred': new_pred_class})
sub.set_index('index', inplace=True)
sub.to_csv('lgbm_sum1.csv')

[1]	valid_0's binary_logloss: 0.668354
Training until validation scores don't improve for 100 rounds
[2]	valid_0's binary_logloss: 0.651005
[3]	valid_0's binary_logloss: 0.63667
[4]	valid_0's binary_logloss: 0.62455
[5]	valid_0's binary_logloss: 0.614548
[6]	valid_0's binary_logloss: 0.60587
[7]	valid_0's binary_logloss: 0.598776
[8]	valid_0's binary_logloss: 0.592769
[9]	valid_0's binary_logloss: 0.587599
[10]	valid_0's binary_logloss: 0.582843
[11]	valid_0's binary_logloss: 0.579072
[12]	valid_0's binary_logloss: 0.575606
[13]	valid_0's binary_logloss: 0.572717
[14]	valid_0's binary_logloss: 0.570267
[15]	valid_0's binary_logloss: 0.567835
[16]	valid_0's binary_logloss: 0.565539
[17]	valid_0's binary_logloss: 0.563932
[18]	valid_0's binary_logloss: 0.562525
[19]	valid_0's binary_logloss: 0.56119
[20]	valid_0's binary_logloss: 0.560301
[21]	valid_0's binary_logloss: 0.559276
[22]	valid_0's binary_logloss: 0.55827
[23]	valid_0's binary_logloss: 0.557639
[24]	valid_0's binary_logloss: 0

In [12]:
#RandomForest결과
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import warnings
import xgboost as xgb
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from sklearn.datasets import load_breast_cancer
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

train2=pd.read_csv("train2.csv")
test=pd.read_csv("test2.csv")

zero_features = ['education', 'engnat', 'hand', 'married']

index = test['index']

pd.set_option('display.max_columns', 100)

Unneed_list = ['QaE', 'QbE', 'QcE', 'QdE', 'QeE','QfE', 'QgE', 'QhE', 'QiE', 'QjE',
               'QkE', 'QlE', 'QmE', 'QnE', 'QoE','QpE', 'QqE', 'QrE', 'QsE', 'QtE', 'index']
train2.drop(Unneed_list, axis=1, inplace=True)
test.drop(Unneed_list, axis=1, inplace=True)


train3=pd.get_dummies(train2)
test=pd.get_dummies(test)


zero_features = ['education', 'engnat', 'hand', 'married']

#결손값(무응답 = 0)제거 함수
def drop_NoResponse(df):
    for col in zero_features:
        df.drop(df[df[col]==0].index, axis=0, inplace=True)
        
    return df
train = drop_NoResponse(train3)



max_index = train['familysize'].idxmax()
train.drop(max_index, axis=0, inplace=True)
#trainX에 투표값 제거
train_X = train.drop('voted', axis = 1)

#trainy에 투표값만 넣음
train_y = train['voted']


X_train, X_test, y_train, y_test=train_test_split(train_X, train_y, test_size=0.2, random_state=0)

rf_clf = RandomForestClassifier(random_state=0)
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))
get_clf_eval(y_test, pred)

X_new = test
X_new.shape
new_pred_class = rf_clf.predict(X_new)
new_pred_class.shape
pred_proba = rf_clf.predict_proba(X_new)[:,1]
sub = pd.DataFrame({'index':index, 'pred_prob':pred_proba, 'pred': new_pred_class})
sub.set_index('index', inplace=True)
sub.to_csv('random_sum.csv')

오차행렬
[[2556 1334]
 [ 848 2409]]
정확도: 0.6947, 정밀도: 0.6436, 재현율: 0.7396, F1: 0.6883
